In [10]:
import spacy
nlp = spacy.load("de_dep_news_trf") #insert the name of the model you want to use (preload if necessary)

c:\Program Files\Python312\Lib\site-packages\thinc\shims\pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_locatio

In [11]:
import functions #functions for normalization and sentence splitting

In [12]:
import pandas as pd

In [13]:
#using the deTenTen23 corpus as an example
df = pd.read_excel('gehören_deTenTen.xlsx') #insert the name of your excel file (or csv file)

In [14]:
# adding id numbers
df['sentence_id'] = df.reset_index().index +1
print(df.count())
print(df.head())

Reference      10000
Sentence       10000
sentence_id    10000
dtype: int64
   Reference                                           Sentence  sentence_id
0    l.de,de                         <s> Dazu gehören z.B. </s>            1
1   fr.de,de  <s> Sie war eine Führungsfigur der frühen Grün...            2
2   ra.de,de  <s> Denn zum Prüfprogramm im Sinn von Art. 59 ...            3
3   mz.de,de  <s> Die Szenen mit ihm gehören zu den wohl rüh...            4
4  t1p.de,de  <s> Zu unserem Service gehören aber mehrere Do...            5


In [15]:
df[['URL','Domain']] = df['Reference'].str.split(',',expand=True).copy()

In [16]:
df.head()

,Reference,Sentence,sentence_id,URL,Domain
0,"l.de,de",<s> Dazu gehören z.B. </s>,1,l.de,de
1,"fr.de,de",<s> Sie war eine Führungsfigur der frühen Grün...,2,fr.de,de
2,"ra.de,de",<s> Denn zum Prüfprogramm im Sinn von Art. 59 ...,3,ra.de,de
3,"mz.de,de",<s> Die Szenen mit ihm gehören zu den wohl rüh...,4,mz.de,de
4,"t1p.de,de",<s> Zu unserem Service gehören aber mehrere Do...,5,t1p.de,de


In [17]:
# Normalize the text in the dataframe
df['text'] = df['Sentence'].apply(functions.normalize_text)
df = df.drop(columns=['Reference', 'Sentence'])
df.head()

,sentence_id,URL,Domain,text
0,1,l.de,de,Dazu gehören z.B.
1,2,fr.de,de,Sie war eine Führungsfigur der frühen Grünen: ...
2,3,ra.de,de,Denn zum Prüfprogramm im Sinn von Art. 59 Satz...
3,4,mz.de,de,Die Szenen mit ihm gehören zu den wohl rührend...
4,5,t1p.de,de,Zu unserem Service gehören aber mehrere Domains.


In [18]:
# Define the delimiters
delimiters = [r'[^\w\s]', r'\boder\b', r'\bund\b']

# Apply the function to the dataframe
df = functions.split_and_explode(df, 'text', delimiters)
print(df.head())
print(df.count())

   sentence_id    URL Domain  \
0            1   l.de     de   
0            1   l.de     de   
0            1   l.de     de   
1            2  fr.de     de   
1            2  fr.de     de   

                                                text  \
0                                  Dazu gehören z.B.   
0                                  Dazu gehören z.B.   
0                                  Dazu gehören z.B.   
1  Sie war eine Führungsfigur der frühen Grünen: ...   
1  Sie war eine Führungsfigur der frühen Grünen: ...   

                                        text_split_0  \
0                                     Dazu gehören z   
0                                                  B   
0                                                      
1       Sie war eine Führungsfigur der frühen Grünen   
1   Als Verwaltungsrätin der Europäischen Gemeins...   

                                text_split_0_split_1  \
0                                     Dazu gehören z   
0                    

In [19]:
df = df.drop(columns=['text_split_0', 'text_split_0_split_1']) #adjust if you have more  or less delimiters

In [20]:
#pre-filtering
df['gehören_yes']=df['final_split'].str.contains(r'geh(ö|oe|o)r*') #adjust the regex to your phenomenon
df = df[df['gehören_yes']]

print(df.count)

<bound method DataFrame.count of       sentence_id                        URL  Domain  \
0               1                       l.de      de   
1               2                      fr.de      de   
2               3                      ra.de      de   
3               4                      mz.de      de   
4               5                     t1p.de      de   
...           ...                        ...     ...   
9995         9996  content-marketing-star.de      de   
9996         9997               blogspot.com     com   
9997         9998               blogspot.com     com   
9998         9999  versicherungsprofi.online  online   
9999        10000       opensourceecology.de      de   

                                                   text  \
0                                     Dazu gehören z.B.   
1     Sie war eine Führungsfigur der frühen Grünen: ...   
2     Denn zum Prüfprogramm im Sinn von Art. 59 Satz...   
3     Die Szenen mit ihm gehören zu den wohl rührend...   

C:\Users\mattesc98\AppData\Local\Temp\ipykernel_9024\3484620260.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['gehören_yes']=df['final_split'].str.contains(r'geh(ö|oe|o)r*') #adjust the regex to your phenomenon


In [21]:
df = df.drop_duplicates(subset='sentence_id')

In [22]:
#filter sentences for the target
target_lemma = 'gehören'        #adjust the lemma to your target phenomenon
target_tag = 'VVPP'             #adjust the tag to your target phenomenon

In [23]:
# extract lemma
def extract_lemma(text):
    doc = nlp(text)
    lemma = [token.text for token in doc if token.lemma_ == target_lemma]
    return ', '.join(lemma)

# Apply the function to the column and create a new column with the extracted verbs
df['gehören'] = df['final_split'].apply(extract_lemma)

c:\Program Files\Python312\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [24]:
# extract tagged verbs
def extract_tag(text):
    doc = nlp(text)
    tags = [token.text for token in doc if token.tag_ == target_tag and token.lemma_ != target_lemma]
    return ', '.join(tags)

# Apply the function to the column and create a new column with the extracted verbs
df['VVPP'] = df['final_split'].apply(extract_tag)

c:\Program Files\Python312\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [25]:
#nan values the df
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.head()

,sentence_id,URL,Domain,text,final_split,gehören_yes,gehören,VVPP
0,1,l.de,de,Dazu gehören z.B.,Dazu gehören z,True,gehören,NaN
1,2,fr.de,de,Sie war eine Führungsfigur der frühen Grünen: ...,gehörte der ersten Fraktionsführung an,True,gehörte,NaN
2,3,ra.de,de,Denn zum Prüfprogramm im Sinn von Art. 59 Satz...,2 BayBO gehören ausschließlich vom Bauherrn t...,True,gehören,NaN
3,4,mz.de,de,Die Szenen mit ihm gehören zu den wohl rührend...,Die Szenen mit ihm gehören zu den wohl rührend...,True,gehören,NaN
4,5,t1p.de,de,Zu unserem Service gehören aber mehrere Domains.,Zu unserem Service gehören aber mehrere Domains,True,gehören,NaN


In [26]:
#remove the rows with NaN
df.dropna(subset = ['gehören', 'VVPP'], inplace =True)
df.head()

,sentence_id,URL,Domain,text,final_split,gehören_yes,gehören,VVPP
234,235,mademyday.com,com,Du gehörst dann zu den vielen Leuten die sich ...,Du gehörst dann zu den vielen Leuten die sich ...,True,gehörst,informiert
374,375,felsenheimat.de,de,Hierzu nutze ich am liebsten die seit Jahren v...,Aktuelle Wetterdaten gehören ebenfalls mit ei...,True,gehören,einbezogen
419,420,blurayreviews.ch,ch,Mit 'Blackhat' hat Michael Mann einen zwar zei...,Thriller geschaffen der leider zu den schwäche...,True,gehört,geschaffen
427,428,philhaarmonie.de,de,Wetten auf Sportereignisse wo vorwiegend minde...,gehören verboten,True,gehören,verboten
526,527,kredit-journal.net,net,Die Deutsche Kreditbank AG wurde bereits 1990 ...,Die Deutsche Kreditbank AG wurde bereits 1990 ...,True,gehört,gegründet


In [27]:
df.count()

sentence_id    145
URL            145
Domain         145
text           145
final_split    145
gehören_yes    145
gehören        145
VVPP           145
dtype: int64

In [28]:
print(df)
df.to_excel('gehören_passive_filtered.xlsx', index=False)

      sentence_id                   URL Domain  \
234           235         mademyday.com    com   
374           375       felsenheimat.de     de   
419           420      blurayreviews.ch     ch   
427           428      philhaarmonie.de     de   
526           527    kredit-journal.net    net   
...           ...                   ...    ...   
9785         9786     bottwartalbahn.de     de   
9789         9790     adventureinsel.de     de   
9842         9843    vw-golf-country.de     de   
9892         9893  tierregistrierung.de     de   
9982         9983                w4f.eu     eu   

                                                   text  \
234   Du gehörst dann zu den vielen Leuten die sich ...   
374   Hierzu nutze ich am liebsten die seit Jahren v...   
419   Mit 'Blackhat' hat Michael Mann einen zwar zei...   
427   Wetten auf Sportereignisse wo vorwiegend minde...   
526   Die Deutsche Kreditbank AG wurde bereits 1990 ...   
...                                          

In [30]:
df['Domain'].value_counts()

Domain
de      85
at      21
com     19
net      9
ch       3
org      2
eu       2
info     2
blog     1
news     1
Name: count, dtype: int64

In [31]:
df['VVPP'].value_counts()

VVPP
verboten        9
gesehen         5
abgeschafft     4
diskutiert      3
bestraft        3
               ..
entmachtet      1
verbracht       1
erlegt          1
veranstaltet    1
weggesperrt     1
Name: count, Length: 110, dtype: int64